In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.autograd import Variable
import operator
import warnings
warnings.filterwarnings('ignore')
import os
from queue import PriorityQueue
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np
import pickle as pkl
from sacrebleu import raw_corpus_bleu
import math, copy, time


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
#Define constants here
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2
UNK_TOKEN = 3
BATCH_SIZE = 32
TEST_BATCH_SIZE = 3
words_to_load = 80000
d_model = emb_size = 300
wiki_size = 300
CUDA = True
MAX_LENGTH = 100
hidden_size = 300

In [48]:
PAD_TOKEN = 0
SOS_TOKEN = 1
EOS_TOKEN = 2
UNK_TOKEN = 3
BATCH_SIZE = 3

class Lang:
    def __init__(self, name, emb_word2id, emb_id2word, emb_ordered_words):
        self.name = name
        self.word2index = emb_word2id
        self.word2count = {}
        self.index2word = emb_id2word #Dict
        self.n_words = 4  # Count SOS and EOS +(batch: pad and unk)

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2count:
            self.word2count[word] = 1
            self.n_words += 1
        else:
            self.word2count[word] += 1
            

# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

#Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    #This line is commented out since it will not properly deal with Chinese Letters
    #s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

#reference: LAB4 hw2
def indexesFromSentences(lang1, lang2, pairs):
    id_list1 = []
    id_list2 = []
    for i in range(len(pairs)):
        sentence1 = pairs[i][0]
        sentence2 = pairs[i][1]
        
        sentence1 = sentence1.replace('quot','')
        sentence1 = sentence1.replace('apos', '')
        sentence2 = sentence2.replace('quot','')
        sentence2 = sentence2.replace('apos', '')
        #If either sentence is empty, then remove the pair
        if sentence1 == '' or sentence2 == '':
            continue;
        
        id_sentence1 = [lang1.word2index[word] if word in lang1.word2index else UNK_TOKEN 
                        for word in sentence1.split()] + [EOS_TOKEN]
        id_list1.append(id_sentence1)
        id_sentence2 = [lang2.word2index[word] if word in lang2.word2index else UNK_TOKEN 
                        for word in sentence2.split()] + [EOS_TOKEN]
        id_list2.append(id_sentence2)
        
   
        
    return id_list1,id_list2

In [15]:
#Reference Lab4 HW2
datadir = os.getcwd()
words_to_load = 200000
with open(datadir + '/data/wiki-news-300d-1M.vec') as f:
    loaded_en_embeddings = np.zeros(((words_to_load+4), wiki_size))
    en_word2id = {}
    en_id2words = {}
    
    en_id2words[PAD_TOKEN] = '<PAD>'
    en_id2words[SOS_TOKEN] = '<SOS>'
    en_id2words[EOS_TOKEN] = '<EOS>'
    en_id2words[UNK_TOKEN] = '<UNK>'
    
    en_word2id['<PAD>'] = PAD_TOKEN
    en_word2id['<SOS>'] = SOS_TOKEN
    en_word2id['<EOS>'] = EOS_TOKEN
    en_word2id['<UNK>'] = UNK_TOKEN
    
    en_ordered_words= []
    en_ordered_words.append('<PAD>')
    en_ordered_words.append('<SOS>')
    en_ordered_words.append('<EOS>')
    en_ordered_words.append('<UNK>')
    
    for i, line in enumerate(f):
        if i >= words_to_load:
            break
        if i ==0:#Ignore the first line
            continue;
        s = line.split()
        #print(len(s))
        loaded_en_embeddings[i+4,:] = np.asarray(s[1:])
        en_word2id[s[0]] = i+4 #for extra pad and unk eos and unk
        en_id2words[i+4] = s[0]
        en_ordered_words.append(s[0])

In [8]:
#Reference Lab4 HW2
#Over 200000 loaded words, only 10 has wrong dimensions, simply discarded
words_to_load = 200000
datadir = os.getcwd()
with open(datadir + '/data/wiki.zh.vec') as f:
    loaded_zh_embeddings = np.zeros(((words_to_load+4),wiki_size))
    zh_word2id = {}
    zh_id2words = {}
    
    zh_id2words[PAD_TOKEN] = '<PAD>'
    zh_id2words[SOS_TOKEN] = '<SOS>'
    zh_id2words[EOS_TOKEN] = '<EOS>'
    zh_id2words[UNK_TOKEN] = '<UNK>'
    
    zh_word2id['<PAD>'] = PAD_TOKEN
    zh_word2id['<SOS>'] = SOS_TOKEN
    zh_word2id['<EOS>'] = EOS_TOKEN
    zh_word2id['<UNK>'] = UNK_TOKEN
    
    zh_ordered_words= []
    zh_ordered_words.append('<PAD>')
    zh_ordered_words.append('<SOS>')
    zh_ordered_words.append('<EOS>')
    zh_ordered_words.append('<UNK>')
    
    for i, line in enumerate(f):
        #print(i)
        if i >= words_to_load:
            break;
        if i == 0: #Ignore the first line
            continue;
        s = line.split()
        if len(s) != 301:
            print('Wrong dimension, skip')
            continue;
        loaded_zh_embeddings[i+4,:] = np.asarray(s[1:])
        zh_word2id[s[0]] = i+4 #for extra pad and unk 
        zh_id2words[i+4] = s[0]
        zh_ordered_words.append(s[0])

Wrong dimension, skip
Wrong dimension, skip
Wrong dimension, skip
Wrong dimension, skip
Wrong dimension, skip
Wrong dimension, skip
Wrong dimension, skip
Wrong dimension, skip
Wrong dimension, skip
Wrong dimension, skip


In [9]:
#Reference Lab4 HW2
#Over 200000 loaded words, 58 has wrong dimensions
words_to_load = 200000
datadir = os.getcwd()
with open(datadir + '/data/wiki.vi.vec') as f:
    loaded_vi_embeddings = np.zeros(((words_to_load+4),wiki_size))
    vi_word2id = {}
    vi_id2words = {}
    
    vi_id2words[PAD_TOKEN] = '<PAD>'
    vi_id2words[SOS_TOKEN] = '<SOS>'
    vi_id2words[EOS_TOKEN] = '<EOS>'
    vi_id2words[UNK_TOKEN] = '<UNK>'
    
    vi_word2id['<PAD>'] = PAD_TOKEN
    vi_word2id['<SOS>'] = SOS_TOKEN
    vi_word2id['<EOS>'] = EOS_TOKEN
    vi_word2id['<UNK>'] = UNK_TOKEN
    
    vi_ordered_words= []
    vi_ordered_words.append('<PAD>')
    vi_ordered_words.append('<SOS>')
    vi_ordered_words.append('<EOS>')
    vi_ordered_words.append('<UNK>')
    wrong_dim = 0;
    for i, line in enumerate(f):
        #print(line)
        if i >= words_to_load:
            break;
        if i == 0: #Ignore the first line
            continue;
        s = line.split()
        if len(s) != 301:
            wrong_dim += 1#Skip the wrong dimension one
            continue;
        loaded_vi_embeddings[i+4,:] = np.asarray(s[1:])
        vi_word2id[s[0]] = i+4 #for extra pad and unk 
        vi_id2words[i+4] = s[0]
        vi_ordered_words.append(s[0])
    print('In total {} has wrong dimension, hence skipped'.format(wrong_dim))

In total 58 has wrong dimension, hence skipped


In [42]:
loaded_zh_embeddings = pkl.load(open('./data/zh_embeddings.p', 'rb'))
loaded_en_embeddings = pkl.load(open('./data/en_embeddings.p', 'rb'))
loaded_vi_embeddings = pkl.load(open('./data/vi_embeddings.p', 'rb'))


In [10]:
#loading data
zh_train_pairs_cleaned = pkl.load(open('./data/zh_train_pairs_cleaned.p', 'rb'))
zh_val_pairs_cleaned = pkl.load(open('./data/zh_val_pairs_cleaned.p', 'rb'))
zh_test_pairs_cleaned = pkl.load(open('./data/zh_test_pairs_cleaned.p', 'rb'))

vi_train_pairs_cleaned = pkl.load(open('./data/vi_train_pairs_cleaned.p', 'rb'))
vi_val_pairs_cleaned = pkl.load(open('./data/vi_val_pairs_cleaned.p', 'rb'))
vi_test_pairs_cleaned = pkl.load(open('./data/vi_test_pairs_cleaned.p', 'rb'))

In [49]:
class NMTDataset(Dataset):
    def __init__(self, pairs):#Needs the index pairs
        self.pairs = pairs
#         self.input_lang = input_lang
#         self.output_lang = output_lang
        self.input_seqs = [pairs[i][0] for i in range(len(self.pairs))]
        self.output_seqs = [pairs[i][1] for i in range(len(self.pairs))]

    def __len__(self):
        return len(self.pairs)#Returning number of pairs
    
    def __getitem__(self, index):
        input_seq = self.input_seqs[index]
        output_seq = self.output_seqs[index]
        return [input_seq, len(input_seq), output_seq, len(output_seq)]
    
def vocab_collate_func(batch):
    #Reference: lab8_3_mri
    def _pad_sequences(seqs):
        lens = [len(seq) for seq in seqs]
        padded_seqs = torch.zeros(len(seqs), max(lens)).long()
        for i, seq in enumerate(seqs):
            end = lens[i]
            padded_seqs[i, :end] = torch.LongTensor(seq[:end])
        return padded_seqs, lens
    
    batch_input_seqs = [datum[0] for datum in batch]
    batch_output_seqs = [datum[2] for datum in batch]
    #batch_input_length = [datum[1] for datum in batch]
    #batch_output_length = [datum[3] for datum in batch]

    sorted_pairs = sorted(zip(batch_input_seqs, batch_output_seqs), key=lambda x: len(x[0]), reverse = True)
    in_seq_sorted, out_seq_sorted = zip(*sorted_pairs)
    
    padded_input,input_lens = _pad_sequences(in_seq_sorted)
    padded_output,output_lens = _pad_sequences(out_seq_sorted)
    
    input_list = torch.from_numpy(np.array(padded_input))
    input_length = torch.LongTensor(input_lens)
    output_list = torch.from_numpy(np.array(padded_output))
    output_length = torch.LongTensor(output_lens)
    
    if CUDA:
        input_list = input_list.cuda()
        output_list = output_list.cuda()
        input_length = input_length.cuda()
        output_length = out_length.cuda()
            
    return [input_list, input_length, output_list, output_length]

In [50]:
BATCH_SIZE = 32
'''
NMTDataset needs index pairs, need to call indexesFromPairs functions beforehand
The dataLoader is sorted according to length of the input_length, and padded to
max length of input and output list repectively
TODO: output_list is not sorted, hence need to sort (maybe) in the rnn sequence.
'''
# train_zh_dataset = NMTDataset(zh_train_pairs_cleaned, input_zh, output_zh_en)
# train_vi_dataset = NMTDataset(vi_train_pairs_cleaned, input_vi, output_vi_en)
# val_zh_dataset = NMTDataset(zh_val_pairs_cleaned, input_zh, output_zh_en)
# val_vi_dataset = NMTDataset(vi_val_pairs_cleaned, input_vi, output_vi_en)
# test_zh_dataset = NMTDataset(zh_test_pairs_cleaned, input_zh, output_zh_en)
# test_vi_dataset = NMTDataset(vi_test_pairs_cleaned, input_vi, output_vi_en)

train_zh_dataset = NMTDataset(zh_train_pairs_cleaned)
train_vi_dataset = NMTDataset(vi_train_pairs_cleaned)
val_zh_dataset = NMTDataset(zh_val_pairs_cleaned)
val_vi_dataset = NMTDataset(vi_val_pairs_cleaned)
test_zh_dataset = NMTDataset(zh_test_pairs_cleaned)
test_vi_dataset = NMTDataset(vi_test_pairs_cleaned)


train_zh_loader = torch.utils.data.DataLoader(dataset = train_zh_dataset, 
                                          batch_size = BATCH_SIZE,
                                          collate_fn = vocab_collate_func,
                                          shuffle = True)

train_vi_loader = torch.utils.data.DataLoader(dataset = train_vi_dataset, 
                                          batch_size = BATCH_SIZE,
                                          collate_fn = vocab_collate_func,
                                          shuffle = True)

#Will use batch size 1 for validation and test since the sentence will be translated one by one
val_zh_loader = torch.utils.data.DataLoader(dataset = val_zh_dataset, 
                                          batch_size = 1,
                                          collate_fn = vocab_collate_func,
                                          shuffle = False)
val_vi_loader = torch.utils.data.DataLoader(dataset = val_vi_dataset, 
                                          batch_size = 1,
                                          collate_fn = vocab_collate_func,
                                          shuffle = False)
test_zh_loader = torch.utils.data.DataLoader(dataset = test_zh_dataset, 
                                          batch_size = 1,
                                          collate_fn = vocab_collate_func,
                                          shuffle = False)
test_vi_loader = torch.utils.data.DataLoader(dataset = test_vi_dataset, 
                                          batch_size = 1,
                                          collate_fn = vocab_collate_func,
                                          shuffle = False)
#Input_batch in size Batch x maxLen


In [51]:
for i, (input_list, input_length, output_list, output_length) in enumerate(train_zh_loader):
    if i== 0:
        break

In [52]:
input_list.size()

torch.Size([32, 42])

In [53]:
#Here for the constant definition
MAX_SENTENCE_LENGTH = 10
hidden_size = 300
max_length = 10
BATCH_SIZE = 3
TEST_BATCH_SIZE = 3
CLIP = 50
TEACHER_RATIO = 0.5
EN_ORDERED_NUM = 200003
ZH_ORDERED_NUM = 199945
VI_ORDERED_NUM = 199993


In [54]:
class Embeddings(nn.Module):
    def __init__(self, emb, d_model, vocab):
        super(Embeddings, self).__init__()
        self.emb = emb
        self.lut = nn.Embedding.from_pretrained(self.emb, False, False)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [55]:
#Reference: https://stackoverflow.com/questions/52922445/runtimeerror-exp-not-implemented-for-torch-longtensor
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
       
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

In [56]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [57]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [58]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [59]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [60]:
def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

In [61]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [62]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [63]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
        
    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=mask, 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [64]:
class SelfEncoder(nn.Module):
    """
    A self attention based Encoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, src_embed):
        super(SelfEncoder, self).__init__()
        self.encoder = encoder
        #self.decoder = decoder
        self.src_embed = src_embed
        #self.tgt_embed = tgt_embed
        #self.generator = generator
        
    def forward(self, src, src_mask=None):
        "Take in and process masked src and target sequences."
        return self.encoder(self.src_embed(src), src_mask)
    

In [65]:
def make_model(emb, src_vocab, N=6, 
               d_model=300, d_ff=2048, h=6, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = SelfEncoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(emb, d_model, src_vocab), c(position)))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [66]:
#Reference: https://github.com/spro/practical-pytorch/blob/master/seq2seq-translation/seq2seq-translation-batched.ipynb
#Reference: lab8 1_nmt, lab8 3_mri
class PreAttnDecoderRNN(nn.Module):
    def __init__(self, emb, emb_size, hidden_size, output_size, n_layers=1, dropout_p=0.1):
        super(PreAttnDecoderRNN, self).__init__()
        self.emb = emb
        self.emb_size = emb_size
        self.hidden_size = hidden_size
        self.output_size = output_size#vocab size of the output lang
        self.n_layers = n_layers
        self.dropout = nn.Dropout(dropout_p)

        self.embedding = nn.Embedding.from_pretrained(self.emb, False, False,)
        
        #self.attn = nn.Linear(hidden_size*, hidden_size)
        #self.attn2 = nn.Linear(hidden_size, hidden_size)
#         self.v = nn.Parameter(torch.rand(hidden_size))
        self.concat = nn.Linear(2*hidden_size, hidden_size)
        self.gru = nn.GRU(emb_size, hidden_size, n_layers, dropout = dropout_p)
        self.out = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, word_input, last_hidden, encoder_outputs):
        #Here encoder_outputs is in size batch x max_seq x d_model
        self.batch_size = encoder_outputs.size(0)
        max_len = encoder_outputs.size(1)
        attn_energies = Variable(torch.zeros(self.batch_size, max_len))#B X max_len
        attn_energies = attn_energies.cuda() if CUDA else attn_energies
        
        
        embedded = self.embedding(word_input)
        embedded = self.dropout(embedded)
        embedded = embedded.view(1, self.batch_size, -1) # S=1 x B x N
        
        # Get current hidden state from input word and last hidden state
        rnn_output, hidden = self.gru(embedded, last_hidden)
        #rnn layer x batch x h
        #encoder-outputs  max_len x batch x h
        
#         for b in range(self.batch_size):
#             # Calculate energy for each encoder output
#             for i in range(max_len):
#                 attn_energies[b, i] = (rnn_output[:, b].squeeze()).dot(encoder_outputs[i, b])
        
        # Calculate attention from current RNN state and all encoder outputs;
        #More efficient
        #attn_energies = ((rnn_output.transpose(0,1)).bmm(encoder_outputs.transpose(0,1).transpose(1,2))).squeeze(1)
        attn_energies = ((rnn_output.transpose(0,1)).bmm(encoder_outputs.transpose(1,2))).squeeze(1)

        attn_weights = F.softmax(attn_energies) # B x max_len
        attn_weights = attn_weights.unsqueeze(1)
        # apply to encoder outputs to get weighted average
        context = attn_weights.bmm(encoder_outputs) # B x S=1 x N
        
        # Attentional vector using the RNN hidden state and context vector

        # concatenated together (Luong eq. 5)
        rnn_output = rnn_output.squeeze(0) # S=1 x B x N -> B x N
        context = context.squeeze(1)       # B x S=1 x N -> B x N
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = F.tanh(self.concat(concat_input))

        # Finally predict next token (Luong eq. 6, without softmax)
        output = self.out(concat_output)
        output = F.log_softmax(output)


        # Return final output, hidden state, and attention weights (for visualization)
        return output, hidden, attn_weights

### Test for decoder PLS ignore

In [67]:
curr_batch = input_list.size(0)
decoder_input = torch.tensor([[SOS_TOKEN]]*curr_batch, device=device)
    
#decoder_hidden = encoder_hidden[:batch_decoder.n_layers]#Bidirectional summoned
decoder_hidden = torch.zeros(attn_decoder.n_layers, curr_batch, attn_decoder.hidden_size)


In [68]:
batch_size = encoder_outputs.size(0)
max_len = encoder_outputs.size(1)
        

In [137]:
attn_energies = Variable(torch.zeros(batch_size, max_len))#B X max_len


In [150]:
attn_weights = F.softmax(attn_energies)

In [153]:
attn_weights = attn_weights.unsqueeze(1)

In [155]:
attn_weights.size(), encoder_outputs.size()

(torch.Size([32, 1, 56]), torch.Size([32, 56, 300]))

In [156]:
context = attn_weights.bmm(encoder_outputs)

In [157]:
context.size()

torch.Size([32, 1, 300])

In [158]:
rnn_output = rnn_output.squeeze(0)

In [159]:
context = context.squeeze(1)  

In [160]:
rnn_output.size()

torch.Size([32, 300])

In [139]:
embedded = loaded_en_embeddings[decoder_input]

embedded = embedded.view(1, batch_size, -1)

In [140]:
embedded.size()

torch.Size([1, 32, 300])

In [142]:
rnn_output, hidden = attn_decoder.gru(embedded, decoder_hidden)

In [143]:
encoder_outputs.size()

torch.Size([32, 56, 300])

In [144]:
for b in range(batch_size):
# Calculate energy for each encoder output
    for i in range(max_len):
        attn_energies[b, i] = (rnn_output[:, b].squeeze()).dot(encoder_outputs[b, i])

In [145]:
temp_test = ((rnn_output.transpose(0,1)).bmm(encoder_outputs.transpose(1,2))).squeeze(1)

In [147]:
torch.allclose(temp_test, attn_energies)

True

In [161]:
concat_input = torch.cat((rnn_output, context), 1)

In [162]:
concat_input.size()

torch.Size([32, 600])

In [164]:
concat_output = F.tanh(attn_decoder.concat(concat_input))

In [165]:
concat_output.size()

torch.Size([32, 300])

In [167]:
output = attn_decoder.out(concat_output)
output = F.log_softmax(output)

### End of decode test

In [33]:
CUDA = False
loaded_zh_embeddings = torch.from_numpy(loaded_zh_embeddings).float()
loaded_vi_embeddings = torch.from_numpy(loaded_vi_embeddings).float()
loaded_en_embeddings = torch.from_numpy(loaded_en_embeddings).float()

if CUDA:
    loaded_zh_embeddings = loaded_zh_embeddings.cuda()
    loaded_vi_embeddings = loaded_vi_embeddings.cuda()
    loaded_en_embeddings = loaded_en_embeddings.cuda()

In [69]:
#Example of encoder and decoder
self_encoder = make_model(loaded_zh_embeddings, len(zh_ordered_words)).to(device)
attn_decoder = PreAttnDecoderRNN(loaded_en_embeddings, emb_size, hidden_size, len(en_ordered_words)).to(device)

#SLOW
learning_rate = 0.01
encoder_optimizer = optim.Adam(self_encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(attn_decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [71]:
encoder_outputs = self_encoder(input_list)
encoder_outputs.size()

torch.Size([32, 42, 300])

In [79]:
loss = self_attn_train(input_list, output_list, output_length, self_encoder, attn_decoder, 
                      encoder_optimizer, decoder_optimizer, criterion)

In [78]:
#Referenced from lab8 1nmt and modified 
CUDA = False
def self_attn_train(input_list, output_list,output_length, 
                self_encoder, attn_decoder, encoder_optimizer, decoder_optimizer, criterion):
    '''
    param: @attention is a Boolean variable indicating whether using attention
    '''
    self_encoder.train()
    attn_decoder.train()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    max_output_length = output_length.max().item()
    
    loss = 0

    encoder_outputs = self_encoder(input_list)

    #Initialize for decoding process
    curr_batch = input_list.size(0)#Take the current batch size
    decoder_input = torch.tensor([[SOS_TOKEN]]*curr_batch, device=device)
    
    #decoder_hidden = encoder_hidden[:batch_decoder.n_layers]#Bidirectional summoned
    decoder_hidden = torch.zeros(attn_decoder.n_layers, curr_batch, attn_decoder.hidden_size)
    decoder_outputs = torch.zeros(max_output_length, curr_batch, attn_decoder.output_size)
    
    # Move new Variables to CUDA
    if CUDA:
        decoder_input = decoder_input.cuda()
        decoder_outputs = decoder_outputs.cuda()
    
    #use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
    use_teacher_forcing = True
    
    if use_teacher_forcing:
    # Teacher forcing: Feed the target as the next input
        for di in range(max_output_length):
            decoder_output, decoder_hidden, decoder_attention = attn_decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_outputs[di] = decoder_output
            decoder_input = output_list[:,di] # Teacher forcing
            loss += criterion(decoder_output, output_list[:,di].contiguous())

    else:
    # Without teacher forcing: use its own predictions as the next input
        for di in range(max_output_length):
            decoder_output, decoder_hidden, decoder_attention = attn_decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            
            decoder_outputs[di] = decoder_input
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()# detach from history as input: size batch x 1 
            loss += criterion(decoder_output, output_list[:,di].contiguous())
            
    #loss += rnn_mask_loss(decoder_outputs.transpose(0,1).contiguous(), output_list.contiguous(), output_length)
            
    loss.backward()
    #ec = torch.nn.utils.clip_grad_norm(batch_encoder.parameters(), CLIP)
    #dc = torch.nn.utils.clip_grad_norm(batch_decoder.parameters(), CLIP)

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item()/max_output_length

In [82]:
#Reference LAB8 1-nmt
def AttnTrainIters(train_loader, self_encoder, attn_decoder, n_iters, print_every=100, plot_every=100, learning_rate=0.01):
    start = time.time()
    learning_curve_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(self_encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(attn_decoder.parameters(), lr=learning_rate)
    
    criterion = nn.NLLLoss()
    for i in range(n_iters):
        #print("Epoch {}/{}".format(i+1, n_epochs))
        losses = []
        for i, (input_list,input_length,output_list, output_length) in enumerate(train_loader):
            loss = self_attn_train(input_list, output_list, output_length, self_encoder, attn_decoder, 
                      encoder_optimizer, decoder_optimizer, criterion)
        
            print_loss_total += loss
            plot_loss_total += loss
            learning_curve_losses.append(loss)

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            
    torch.save(self_encoder.state_dict(), model_path + "self_encoder.pth")
    torch.save(attn_decoder.state_dict(), model_path + "self_attn_decoder.pth")

    showPlot(plot_losses)

In [84]:
AttnTrainIters(train_zh_loader, self_encoder, attn_decoder, 100, print_every=100, plot_every=100, learning_rate=0.01)

KeyboardInterrupt: 

In [83]:
#Reference lab8 1-nmt
def greedy_selfattn_evaluate(val_loader, encoder, decoder, en_id2words):
    #Will generate sentences 1 by 1. 
    # process input sentence
    decoded_words_all = []
    decoder_attentions_all = []
    
    with torch.no_grad():
        encoder.eval()
        decoder.eval()
        
        for i, (input_list, input_length, output_list, output_length) in enumerate(val_loader):
            #if i == 5:
            #    break
            #batch_size, max_len = output_list.size()
            #print(input_list.size())
            
            # encode the source lanugage
            encoder_outputs = encoder(input_list)

            decoder_input = torch.tensor([[SOS_TOKEN]], device=device)  # SOS
            decoder_hidden = torch.zeros(attn_decoder.n_layers, curr_batch, attn_decoder.hidden_size)

            # decode the context vector
            
            # output of this function
            decoded_words = []
            decoder_attentions = torch.zeros(100, 100)

            for di in range(MAX_LENGTH):
                # for each time step, the decoder network takes two inputs: previous outputs and the previous hidden states
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_outputs)

                top_score, topi = decoder_output.data.topk(1)
                decoder_attentions[di, :decoder_attention.size(-1)] = decoder_attention
                decoded_words.append(en_id2words[topi.item()])
                if topi.item() == EOS_TOKEN:
                    break
                else:
                    decoder_input = topi.squeeze().detach()
                    
            decoded_words_all.append(decoded_words)
            decoder_attentions_all.append(decoder_attentions[:di+1])

        return decoded_words_all, decoder_attentions_all